## ChatLab: Chat Experiments, Simplified

💬🔬

In [14]:
import chatlab

conversation = chatlab.Conversation()

conversation.submit("What kind of bird is a murkrow?")

Murkrow is a Pokémon species, rather than a real-life bird. It is a Dark/Flying type Pokémon, known for its dark purple coloration, red eyes, and ability to bring bad luck.

In [17]:
from chatlab import Conversation, user, system

conversation = Conversation()

conversation.submit(
    system("You are a data science tutor running inside of a Jupyter "
           "Notebook environment that has the full NumFOCUS/PyData stack."),
    user("Help me work with circulation-grade-6.csv"),
)

Certainly! Let's start by loading the dataset `circulation-grade-6.csv` into a Pandas DataFrame.

First, make sure that the CSV file is in the same directory as your Jupyter Notebook. Then, you can use the following code to load the dataset:

```python
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv("circulation-grade-6.csv")

# Display the first few rows of the DataFrame
df.head()
```

This code imports the `pandas` library and uses the `read_csv()` function to load the CSV file into a DataFrame called `df`. The `head()` function is then used to display the first few rows of the DataFrame.

Once you have loaded the dataset, you can perform various operations on it, such as exploring the data, analyzing it, or visualizing it. Please let me know what specific task or operation you would like assistance with.

## Introducing Chat Functions

Where `chatlab` (and OpenAI, really) take it next level is with _Chat Functions_. You can declare a function with a schema then register it in your `Conversation` for GPT Models to call. You may recall this kind of behavior from [ChatGPT Plugins](https://noteable.io/chatgpt-plugin-for-notebook/). Now, you can take this even further with your own custom code.

As an example, let's give the large language models the ability to tell time.

In [4]:
from datetime import datetime
from pytz import timezone, all_timezones, utc
from typing import Optional
from pydantic import BaseModel

def what_time(tz: Optional[str] = None):
    '''Current time, defaulting to UTC'''
    if tz is None:
        pass
    elif tz in all_timezones:
        tz = timezone(tz)
    else:
        return 'Invalid timezone'
        
    return datetime.now(tz).strftime('%I:%M %p')

class WhatTime(BaseModel):
    tz: Optional[str]

Let's break this down.

`what_time` is the function we're going to provide access to. Its docstring forms the `description` for the model while the schema comes from the pydantic `BaseModel` called `WhatTime`.

In [5]:
import chatlab

conversation = chatlab.Conversation()

# Register our function
conversation.register(what_time, WhatTime)
# For ease of reuse
chat = conversation.submit

In [6]:
chat("What time is it?")

The current time is 03:06 PM.

In [7]:
conversation.messages

[{'role': 'user', 'content': 'What time is it?'},
 {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'what_time', 'arguments': '{}'}},
 {'role': 'function', 'content': "'03:06 PM'", 'name': 'what_time'},
 {'role': 'assistant', 'content': 'The current time is 03:06 PM.'}]

In [8]:
chat("Great. What time is it in Osaka?")

The current time in Osaka is 07:06 AM.

Under the hood, `chatlab` is keeping track of the conversation history for you in `.messages`

In [9]:
conversation.messages

[{'role': 'user', 'content': 'What time is it?'},
 {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'what_time', 'arguments': '{}'}},
 {'role': 'function', 'content': "'03:06 PM'", 'name': 'what_time'},
 {'role': 'assistant', 'content': 'The current time is 03:06 PM.'},
 {'role': 'user', 'content': 'Great. What time is it in Osaka?'},
 {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'what_time',
   'arguments': '{\n  "tz": "Asia/Tokyo"\n}'}},
 {'role': 'function', 'content': "'07:06 AM'", 'name': 'what_time'},
 {'role': 'assistant', 'content': 'The current time in Osaka is 07:06 AM.'}]

## Outro

Thanks for reading through! Please post issues and reach out. I want to hear your experiences!